# Pytorch Workflow Fundamentals

1: "data (prepare and load)",

2: "build model",

3: "fitting the model to data (training)",

4: "making predictions and evaluating a model (inference)",

5: "saving and loading a model",

6: "putting it all together"


## Data Preparation
**Making data into tensors**

In [1]:
import torch
from torch import nn
## nn has all building blocks of pytorch, like layers, loss functions, etc.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split

torch.__version__

'2.0.1+cu117'

In [2]:
data = pd.read_csv('Sleep_health_and_lifestyle_dataset.csv')
data.sample(10)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
150,151,Female,39,Accountant,8.0,9,80,3,Normal Weight,115/78,67,7500,None
93,94,Male,35,Lawyer,7.4,7,60,5,Obese,135/88,84,3300,Sleep Apnea
122,123,Female,37,Accountant,7.2,8,60,4,Normal,115/75,68,7000,None
222,223,Male,44,Salesperson,6.3,6,45,7,Overweight,130/85,72,6000,Insomnia
353,354,Female,58,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
268,269,Female,49,Nurse,6.0,6,90,8,Overweight,140/95,75,10000,Sleep Apnea
254,255,Female,45,Teacher,6.5,7,45,4,Overweight,135/90,65,6000,Insomnia
328,329,Female,53,Engineer,8.3,9,30,3,Normal,125/80,65,5000,None
65,66,Male,32,Doctor,6.2,6,30,8,Normal,125/80,72,5000,None
212,213,Male,43,Engineer,7.8,8,90,5,Normal,130/85,70,8000,None


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Person ID                374 non-null    int64  
 1   Gender                   374 non-null    object 
 2   Age                      374 non-null    int64  
 3   Occupation               374 non-null    object 
 4   Sleep Duration           374 non-null    float64
 5   Quality of Sleep         374 non-null    int64  
 6   Physical Activity Level  374 non-null    int64  
 7   Stress Level             374 non-null    int64  
 8   BMI Category             374 non-null    object 
 9   Blood Pressure           374 non-null    object 
 10  Heart Rate               374 non-null    int64  
 11  Daily Steps              374 non-null    int64  
 12  Sleep Disorder           374 non-null    object 
dtypes: float64(1), int64(7), object(5)
memory usage: 38.1+ KB


In [4]:
encode1 = []
encode2 = []
for col in data.columns:
    if data[col].dtype == 'object':
        if len(data[col].unique()) == 2:
            encode1.append(col)
            print(col, data[col].unique())
        else:
            encode2.append(col)
            print(col, data[col].unique())
print("\nLinear encoding",encode1)
encode2.remove("Sleep Disorder")
encode2.remove("Blood Pressure")
print("One hot encoding",encode2)

Gender ['Male' 'Female']
Occupation ['Software Engineer' 'Doctor' 'Sales Representative' 'Teacher' 'Nurse'
 'Engineer' 'Accountant' 'Scientist' 'Lawyer' 'Salesperson' 'Manager']
BMI Category ['Overweight' 'Normal' 'Obese' 'Normal Weight']
Blood Pressure ['126/83' '125/80' '140/90' '120/80' '132/87' '130/86' '117/76' '118/76'
 '128/85' '131/86' '128/84' '115/75' '135/88' '129/84' '130/85' '115/78'
 '119/77' '121/79' '125/82' '135/90' '122/80' '142/92' '140/95' '139/91'
 '118/75']
Sleep Disorder ['None' 'Sleep Apnea' 'Insomnia']

Linear encoding ['Gender']
One hot encoding ['Occupation', 'BMI Category']


In [5]:
data[["Systolic BP", "Diastolic BP"]] = data['Blood Pressure'].str.split('/', expand=True)
data["Systolic BP"] = data["Systolic BP"].astype('int')
data["Diastolic BP"] = data["Diastolic BP"].astype('int')
data.drop('Blood Pressure',axis=1,inplace=True)

data.sample(5)

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder,Systolic BP,Diastolic BP
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,77,4200,None,126,83
15,16,Male,29,Doctor,6.0,6,30,8,Normal,70,8000,None,120,80
87,88,Male,35,Engineer,7.2,8,60,4,Normal,65,5000,None,125,80
132,133,Male,38,Lawyer,7.3,8,60,5,Normal,68,8000,None,130,85
226,227,Female,44,Teacher,6.6,7,45,4,Overweight,65,6000,Insomnia,135,90


In [6]:
data = pd.get_dummies(data, columns=encode2, drop_first=True)
data.sample(5)

,Person ID,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Sleep Disorder,...,Occupation_Manager,Occupation_Nurse,Occupation_Sales Representative,Occupation_Salesperson,Occupation_Scientist,Occupation_Software Engineer,Occupation_Teacher,BMI Category_Normal Weight,BMI Category_Obese,BMI Category_Overweight
40,41,Male,31,7.7,7,75,6,70,8000,None,...,0,0,0,0,0,0,0,0,0,0
23,24,Male,30,7.7,7,75,6,70,8000,None,...,0,0,0,0,0,0,0,0,0,0
89,90,Male,35,7.3,8,60,4,65,5000,None,...,0,0,0,0,0,0,0,0,0,0
45,46,Male,31,7.8,7,75,6,70,8000,None,...,0,0,0,0,0,0,0,0,0,0
84,85,Male,35,7.5,8,60,5,70,8000,None,...,0,0,0,0,0,1,0,1,0,0


In [7]:
data = pd.get_dummies(data, columns=encode1, drop_first=True)
data.sample(5)

,Person ID,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Sleep Disorder,Systolic BP,...,Occupation_Nurse,Occupation_Sales Representative,Occupation_Salesperson,Occupation_Scientist,Occupation_Software Engineer,Occupation_Teacher,BMI Category_Normal Weight,BMI Category_Obese,BMI Category_Overweight,Gender_Male
168,169,41,7.1,7,55,6,72,6000,None,125,...,0,0,0,0,0,0,0,0,1,1
344,345,57,8.2,9,75,3,68,7000,Sleep Apnea,140,...,1,0,0,0,0,0,0,0,1,0
59,60,32,7.7,7,75,6,70,8000,None,120,...,0,0,0,0,0,0,0,0,0,1
292,293,50,6.1,6,90,8,75,10000,Sleep Apnea,140,...,1,0,0,0,0,0,0,0,1,0
114,115,37,7.2,8,60,4,68,7000,None,115,...,0,0,0,0,0,0,0,0,0,0


In [8]:
data.drop('Person ID',axis=1,inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 24 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Age                              374 non-null    int64  
 1   Sleep Duration                   374 non-null    float64
 2   Quality of Sleep                 374 non-null    int64  
 3   Physical Activity Level          374 non-null    int64  
 4   Stress Level                     374 non-null    int64  
 5   Heart Rate                       374 non-null    int64  
 6   Daily Steps                      374 non-null    int64  
 7   Sleep Disorder                   374 non-null    object 
 8   Systolic BP                      374 non-null    int64  
 9   Diastolic BP                     374 non-null    int64  
 10  Occupation_Doctor                374 non-null    uint8  
 11  Occupation_Engineer              374 non-null    uint8  
 12  Occupation_Lawyer     

In [9]:
data.describe()

,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Systolic BP,Diastolic BP,Occupation_Doctor,...,Occupation_Nurse,Occupation_Sales Representative,Occupation_Salesperson,Occupation_Scientist,Occupation_Software Engineer,Occupation_Teacher,BMI Category_Normal Weight,BMI Category_Obese,BMI Category_Overweight,Gender_Male
count,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,...,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000,374.000000
mean,42.184492,7.132086,7.312834,59.171123,5.385027,70.165775,6816.844920,128.553476,84.649733,0.189840,...,0.195187,0.005348,0.085561,0.010695,0.010695,0.106952,0.056150,0.026738,0.395722,0.505348
std,8.673133,0.795657,1.196956,20.830804,1.774526,4.135676,1617.915679,7.748118,6.161611,0.392699,...,0.396876,0.073029,0.280090,0.103001,0.103001,0.309466,0.230519,0.161533,0.489660,0.500641
min,27.000000,5.800000,4.000000,30.000000,3.000000,65.000000,3000.000000,115.000000,75.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,35.250000,6.400000,6.000000,45.000000,4.000000,68.000000,5600.000000,125.000000,80.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,43.000000,7.200000,7.000000,60.000000,5.000000,70.000000,7000.000000,130.000000,85.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,50.000000,7.800000,8.000000,75.000000,7.000000,72.000000,8000.000000,135.000000,90.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,59.000000,8.500000,9.000000,90.000000,8.000000,86.000000,10000.000000,142.000000,95.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [10]:
data.isnull().sum()

Age                                0
Sleep Duration                     0
Quality of Sleep                   0
Physical Activity Level            0
Stress Level                       0
Heart Rate                         0
Daily Steps                        0
Sleep Disorder                     0
Systolic BP                        0
Diastolic BP                       0
Occupation_Doctor                  0
Occupation_Engineer                0
Occupation_Lawyer                  0
Occupation_Manager                 0
Occupation_Nurse                   0
Occupation_Sales Representative    0
Occupation_Salesperson             0
Occupation_Scientist               0
Occupation_Software Engineer       0
Occupation_Teacher                 0
BMI Category_Normal Weight         0
BMI Category_Obese                 0
BMI Category_Overweight            0
Gender_Male                        0
dtype: int64

In [11]:
# Inspecting Target Variable
data['Sleep Disorder'].value_counts()

# Encoding Target variable
data['Sleep Disorder'] = data['Sleep Disorder'].map({'None':0, 'Insomnia':1, 'Sleep Apnea':2})
data.sample(20)

,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Sleep Disorder,Systolic BP,Diastolic BP,...,Occupation_Nurse,Occupation_Sales Representative,Occupation_Salesperson,Occupation_Scientist,Occupation_Software Engineer,Occupation_Teacher,BMI Category_Normal Weight,BMI Category_Obese,BMI Category_Overweight,Gender_Male
316,53,8.5,9,30,3,65,5000,0,125,80,...,0,0,0,0,0,0,0,0,0,0
275,49,6.2,6,90,8,75,10000,2,140,95,...,1,0,0,0,0,0,0,0,1,0
359,59,8.1,9,75,3,68,7000,0,140,95,...,1,0,0,0,0,0,0,0,1,0
12,29,6.1,6,30,8,70,8000,0,120,80,...,0,0,0,0,0,0,0,0,0,1
245,44,6.5,7,45,4,65,6000,1,135,90,...,0,0,0,0,0,1,0,0,1,0
157,39,7.2,8,60,5,68,8000,0,130,85,...,0,0,0,0,0,0,0,0,0,1
91,35,7.3,8,60,4,65,5000,0,125,80,...,0,0,0,0,0,0,0,0,0,1
241,44,6.3,6,45,7,72,6000,1,130,85,...,0,0,1,0,0,0,0,0,1,1
311,52,6.6,7,45,7,72,6000,1,130,85,...,0,0,0,0,0,0,0,0,1,0
369,59,8.1,9,75,3,68,7000,2,140,95,...,1,0,0,0,0,0,0,0,1,0


In [12]:
data.dtypes

Age                                  int64
Sleep Duration                     float64
Quality of Sleep                     int64
Physical Activity Level              int64
Stress Level                         int64
Heart Rate                           int64
Daily Steps                          int64
Sleep Disorder                       int64
Systolic BP                          int64
Diastolic BP                         int64
Occupation_Doctor                    uint8
Occupation_Engineer                  uint8
Occupation_Lawyer                    uint8
Occupation_Manager                   uint8
Occupation_Nurse                     uint8
Occupation_Sales Representative      uint8
Occupation_Salesperson               uint8
Occupation_Scientist                 uint8
Occupation_Software Engineer         uint8
Occupation_Teacher                   uint8
BMI Category_Normal Weight           uint8
BMI Category_Obese                   uint8
BMI Category_Overweight              uint8
Gender_Male

### Data split

In [13]:
y = data['Sleep Disorder']
X = data.drop('Sleep Disorder',axis=1)

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=69, stratify=y)


print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


# Converting to tensors
X_train = torch.FloatTensor(X_train.values)
X_test = torch.FloatTensor(X_test.values)
y_train = torch.LongTensor(y_train.values)
y_test = torch.LongTensor(y_test.values)


print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


(280, 23) (94, 23) (280,) (94,)
torch.Size([280, 23]) torch.Size([94, 23]) torch.Size([280]) torch.Size([94])


## Building Model



.

In [ ]:
# # Create a Multiclass Classification Model using nn.Module and nn.Linear
# class ClassificationModel(nn.Module):
#     def __init__(self, input_features=14, hidden1=20, hidden2=20, out_features=3):
#         super().__init__()
#         self.f_connected1 = nn.Linear(input_features, hidden1)
#         self.f_connected2 = nn.Linear(hidden1, hidden2)
#         self.out = nn.Linear(hidden2, out_features)

#     def forward(self, x):
#         x = torch.relu(self.f_connected1(x))
#         x = torch.relu(self.f_connected2(x))
#         x = self.out(x)
#         return x